In [68]:
import boto3
import json 
import csv
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

In [69]:
import boto3
LLAMA_MODEL_ID = "meta.llama3-1-405b-instruct-v1:0"
client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-west-2",
)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4(model, prompts, student_answer, output=None):
    if output is None:
        full_prompt = prompts[0] + student_answer
    else:
        full_prompt = (
            prompts[0] + student_answer + "\n\n" + output + "\n\n" + prompts[1]
        )

    body = json.dumps({
        "prompt": full_prompt,
        "max_gen_len": 512,
        "temperature": 0.0,  # deterministic grading
        "top_p": 1.0,
    })
    response = client.invoke_model(
        modelId=LLAMA_MODEL_ID,
        body=body,
        contentType="application/json",
        accept="application/json",
    )
    raw = response["body"].read().decode("utf-8")
    data = json.loads(raw)
    output_text = data.get("generation", raw)
    return output_text

In [ ]:
question = 'Q10'
model = 'meta.llama3-1-405b-instruct-v1:0'
exp_name = 'base(score_rationale_onequery)'
dataset_path = f'answer/{question}.csv'
output_path = f'llama-output/{question}.csv'

In [71]:
prompts = [open(f'prompt/{exp_name}/{question}.txt').read()]
if exp_name == 'rationale_score_twoqueries':
    prompts.append('Assign a score based on your judgment.')
answers = pd.read_csv(dataset_path, header=0)
answers

,Id,EssaySet,EssayText,EssayScore
0,26626,10,white :: it would make it cooler inside becau...,1
1,27289,10,white :: The darker the paint the more heat i...,1
2,26933,10,dark gray :: this color would deflect heat but...,1
3,26556,10,white :: white reflect the light. when you use...,2
4,27343,10,black :: the sun will be attracted to black a...,1
...,...,...,...,...
95,26302,10,white :: i would say black because it exsord ...,1
96,26479,10,white :: the white lid had the lowest tempritu...,2
97,27279,10,white :: The color white might affect the dog...,1
98,26741,10,black :: Black will be a good color for the do...,2


In [72]:
outputs = []
for i, stu in answers.iterrows():
    output = GPT4(model, prompts, stu.EssayText)
    print(i)
    if exp_name == 'rationale_score_twoqueries':
        outputs.append('\n\n'.join([output, GPT4(model, prompts, stu.EssayText, output)]))
    else:
        outputs.append(output)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [73]:
answers[model] = outputs
answers

,Id,EssaySet,EssayText,EssayScore,meta.llama3-1-405b-instruct-v1:0
0,26626,10,white :: it would make it cooler inside becau...,1,the light gray and dark gray are almost in th...
1,27289,10,white :: The darker the paint the more heat i...,1,for the room.\nFor white a average of 42 degr...
2,26933,10,dark gray :: this color would deflect heat but...,1,it is not as hot at black but it is not as co...
3,26556,10,white :: white reflect the light. when you use...,2,In the doghouse it's better to use white colo...
4,27343,10,black :: the sun will be attracted to black a...,1,and the results show that too\n\n\nRating: \n...
...,...,...,...,...,...
95,26302,10,white :: i would say black because it exsord ...,1,. :::: however that wold make it way to hot. W...
96,26479,10,white :: the white lid had the lowest tempritu...,2,. the black lide made the tempriture in the bo...
97,27279,10,white :: The color white might affect the dog...,1,so this can keep the inside temp a little bit...
98,26741,10,black :: Black will be a good color for the do...,2,It is the best choice to keep the dog warm be...


In [74]:
answers.to_csv(output_path, index=False)